<a href="https://colab.research.google.com/github/kiranbeethoju/Stock-Market-Forecast/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

start = '2004-02-01'
end = '2020-06-25'

tickers = ['BHARTIARTL.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(names)


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


                   ticker        Open   Adj Close
Date                                             
2004-02-03  BHARTIARTL.NS   62.298401   55.406151
2004-02-04  BHARTIARTL.NS   58.993000   56.486176
2004-02-05  BHARTIARTL.NS   60.576801   55.449345
2004-02-06  BHARTIARTL.NS   59.865200   60.395962
2004-02-09  BHARTIARTL.NS   64.731598   60.007133
...                   ...         ...         ...
2020-06-22  BHARTIARTL.NS  575.799988  581.049988
2020-06-23  BHARTIARTL.NS  581.049988  578.799988
2020-06-24  BHARTIARTL.NS  581.000000  557.349976
2020-06-25  BHARTIARTL.NS  553.900024  559.000000
2020-06-26  BHARTIARTL.NS  560.000000  560.400024

[4039 rows x 3 columns]


In [2]:
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]

In [3]:
close.describe()

,y
count,4039.000000
mean,289.853174
std,97.660760
min,54.498901
25%,263.781647
50%,301.534058
75%,346.222427
max,598.799988


In [4]:
!pip install fbprophet

In [25]:
from fbprophet import Prophet
from plotly.graph_objs import *

df = open

def run(df):
  df = df[4000:]
  #df = df[df['ds'].dt.dayofweek < 5]
  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=400)
  future.tail()
  forecast = m.predict(future)
  future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})
  mixed = pd.merge(df,future,on="ds",how="inner")
  mixed.tail(30)
  avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3
  mixed["avg_val"] = ''
  for i in range(0, len(mixed)):
    avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
    mixed['avg_val'].iloc[i] = avg
  mixed = mixed
  import plotly as py

  import plotly.graph_objects as go
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                      mode='lines+markers',
                      name='Predicted Opening Values'))
  fig.show()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                      mode='markers',
                      name='Predicted Opening Avg Values '))

  fig.show()
  return mixed

In [26]:
a = run(open)
b = run(close)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [27]:
a = a.rename(columns={"avg_val":"pred_open_avg","y":"actual_open","yhat_upper_future":"yhat_upper_future_open","yhat_future":"yhat_future_open","yhat_lower_future":"yhat_lower_future_open"})
b = b.rename(columns={"avg_val":"pred_close_avg","y":"actual_close","yhat_upper_future":"yhat_upper_future_close","yhat_future":"yhat_future_close","yhat_lower_future":"yhat_lower_future_close"})


In [28]:
a.columns,b.columns

(Index(['ds', 'actual_open', 'yhat_future_open', 'yhat_lower_future_open',
        'yhat_upper_future_open', 'pred_open_avg'],
       dtype='object'),
 Index(['ds', 'actual_close', 'yhat_future_close', 'yhat_lower_future_close',
        'yhat_upper_future_close', 'pred_close_avg'],
       dtype='object'))

In [29]:
c = pd.merge(a,b,on="ds",how="inner")

In [30]:
c.head()

,ds,actual_open,yhat_future_open,yhat_lower_future_open,yhat_upper_future_open,pred_open_avg,actual_close,yhat_future_close,yhat_lower_future_close,yhat_upper_future_close,pred_close_avg
0,2020-05-04,502.450012,523.990473,506.914667,542.086873,524.331,532.500000,529.147935,511.295628,546.384017,528.943
1,2020-05-05,538.000000,531.392060,512.846949,548.219536,530.82,528.049988,536.535279,519.159207,553.211818,536.302
2,2020-05-06,531.750000,539.196597,521.704473,558.790009,539.897,546.000000,538.421259,520.835318,556.034404,538.43
3,2020-05-07,545.349976,535.702291,519.082689,553.984599,536.257,528.049988,536.264881,518.776715,554.169650,536.404
4,2020-05-08,540.000000,536.371158,519.013683,553.269020,536.218,530.049988,542.799588,524.883591,561.101273,542.928


In [31]:
df = c[["ds","actual_open","pred_open_avg","actual_close","pred_close_avg"]]

In [35]:
df.tail()

,ds,actual_open,pred_open_avg,actual_close,pred_close_avg
34,2020-06-22,575.799988,560.175,581.049988,560.008
35,2020-06-23,581.049988,565.462,578.799988,565.347
36,2020-06-24,581.000000,569.849,557.349976,565.323
37,2020-06-25,553.900024,564.023,559.000000,560.07
38,2020-06-26,560.000000,561.312,560.400024,564.573


In [38]:
df["open_diff"] = df["pred_open_avg"]-df["actual_open"]
df["close_diff"] = df["pred_close_avg"]-df["actual_close"]
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,actual_open,pred_open_avg,actual_close,pred_close_avg,open_diff,close_diff
0,2020-05-04,502.450012,524.331,532.500000,528.943,21.8807,-3.55747
1,2020-05-05,538.000000,530.82,528.049988,536.302,-7.18048,8.25211
2,2020-05-06,531.750000,539.897,546.000000,538.43,8.14703,-7.56967
3,2020-05-07,545.349976,536.257,528.049988,536.404,-9.09345,8.35376
4,2020-05-08,540.000000,536.218,530.049988,542.928,-3.78205,12.8782
5,2020-05-11,533.049988,539.975,536.200012,544.174,6.92528,7.97411
6,2020-05-12,533.000000,547.773,559.849976,552.236,14.7732,-7.61371
7,2020-05-13,568.400024,555.734,555.950012,554.427,-12.6665,-1.52331
8,2020-05-14,551.099976,552.917,539.849976,551.507,1.8173,11.6566
9,2020-05-15,543.500000,552.206,554.099976,557.791,8.70622,3.69086
